In [1]:
import numpy as np
import pandas as pd
import json
import os
import math
import keras as k

%config IPCompleter.greedy=True

Using TensorFlow backend.


In [29]:
def df_col2numpy(df, col_names):
    ser = df.apply(lambda row: np.array([row[col] for col in col_names]).flatten(), axis=1)
    arr = np.array(ser.values.tolist())
    return arr

def df_col2series(df, col_names):
    ser = df.apply(lambda row: np.array([row[col] for col in col_names]).flatten(), axis=1)
    return ser

X_list = []
Y = []
HARD_MAX = 9999
COUNTER = 0
path = '../data'
for root, dirs, files in os.walk(path, topdown=False):
    if COUNTER > HARD_MAX:
            break
    for name in files:
        if root[-6:] == 'blocks':
            print(f'#{COUNTER:5} {root}/{name}')
            try:
                df = pd.read_pickle(os.path.join(root, name))
            except Exception as e:
                print(f'Caught Error: {e}')
                continue
            df['output'] = df_col2series(df, ['output'])
            df['shifted'] = df['output'].shift(1, fill_value=[np.zeros(df['output'].iloc[0].shape)])
            df['times'] = df_col2series(df, ['prev', 'next'])
            df['name'] = f'{root}/{name}'
            
            X_list.append(df)
            COUNTER += 1
        
print(f'Passes {COUNTER:6}/{HARD_MAX:6} hard max')
X = pd.DataFrame(pd.concat(X_list), columns=['shifted', 'output', 'times', 'name', 'time'])
X = X.set_index(['name', 'time'])

X.to_pickle()

#    0 ../data/Shake That - Eminem/blocks/Expert.pkl
#    1 ../data/OK - Madeon/blocks/Expert.pkl
#    2 ../data/God Rest Ye Merry Gentlemen/blocks/ExpertPlus.pkl
#    3 ../data/God Rest Ye Merry Gentlemen/blocks/Expert.pkl
#    4 ../data/God Rest Ye Merry Gentlemen/blocks/Hard.pkl
#    5 ../data/From Dusk Till Dawn/blocks/Expert.pkl
#    6 ../data/Kiss/blocks/Expert.pkl
#    7 ../data/Kiss/blocks/Hard.pkl
#    8 ../data/Chop Suey/blocks/ExpertPlus.pkl
#    9 ../data/Chop Suey/blocks/Expert.pkl
#   10 ../data/Chop Suey/blocks/Hard.pkl
#   11 ../data/The Mother We Share/blocks/Expert.pkl
#   12 ../data/The Mother We Share/blocks/Hard.pkl
#   13 ../data/Are You Gonna Be My Girl/blocks/ExpertPlus.pkl
#   14 ../data/Are You Gonna Be My Girl/blocks/Expert.pkl
#   15 ../data/Are You Gonna Be My Girl/blocks/Hard.pkl
#   16 ../data/American Idiot/blocks/Expert.pkl
#   17 ../data/American Idiot/blocks/Hard.pkl
#   18 ../data/Apollo/blocks/ExpertPlus.pkl
#   19 ../data/Apollo/blocks/Expert.pkl
#

#  159 ../data/Cake/blocks/Hard.pkl
#  160 ../data/Nyaruko MoeyoChaos/blocks/Expert.pkl
#  161 ../data/Nyaruko MoeyoChaos/blocks/Hard.pkl
#  162 ../data/Spectrum/blocks/Expert.pkl
#  163 ../data/Spectrum (Calvin Harris Remix) - Florence + The Machine/blocks/Expert.pkl
#  164 ../data/The World Revolving/blocks/Expert.pkl
#  165 ../data/Imagine Dragons - Warrriors/blocks/Expert.pkl
#  166 ../data/Imagine Dragons - Warrriors/blocks/Hard.pkl
#  167 ../data/EyeOfTheTiger/blocks/Expert.pkl
#  168 ../data/EyeOfTheTiger/blocks/Hard.pkl
#  169 ../data/ukigumo/blocks/ExpertPlus.pkl
#  170 ../data/Solo Camp Recommendation/blocks/Expert.pkl
#  171 ../data/BUBBLES - Tokyo Machine/blocks/Expert.pkl
#  172 ../data/BUBBLES - Tokyo Machine/blocks/Hard.pkl
#  173 ../data/Rainbow Notes/blocks/Expert.pkl
#  174 ../data/Rainbow Notes/blocks/Hard.pkl
#  175 ../data/Im Gonna Be/blocks/Expert.pkl
#  176 ../data/Im Gonna Be/blocks/Hard.pkl
#  177 ../data/KizunaAI mirai/blocks/Expert.pkl
#  178 ../data/KizunaAI

#  316 ../data/Black Shout/blocks/Hard.pkl
#  317 ../data/BTH 5 - How To Save A Life/blocks/ExpertPlus.pkl
#  318 ../data/POP STARS vs MEGALOVANIA/blocks/ExpertPlus.pkl
#  319 ../data/Rugs From Me To You/blocks/ExpertPlus.pkl
#  320 ../data/Rugs From Me To You/blocks/Expert.pkl
#  321 ../data/Rugs From Me To You/blocks/Hard.pkl
#  322 ../data/Internet bitch (PLight Remix)/blocks/ExpertPlus.pkl
#  323 ../data/Internet bitch (PLight Remix)/blocks/Expert.pkl
#  324 ../data/Internet bitch (PLight Remix)/blocks/Hard.pkl
#  325 ../data/Feel Good in Black And Yellow/blocks/Expert.pkl
#  326 ../data/Light It Up/blocks/Expert.pkl
#  327 ../data/Light It Up/blocks/Hard.pkl
#  328 ../data/Becoming/blocks/Expert.pkl
#  329 ../data/When Can I See You Again/blocks/ExpertPlus.pkl
#  330 ../data/When Can I See You Again/blocks/Expert.pkl
#  331 ../data/SO HOT - BLACKPINK/blocks/Expert.pkl
#  332 ../data/Ether Strike/blocks/ExpertPlus.pkl
#  333 ../data/The Kill/blocks/Expert.pkl
#  334 ../data/The Kil

#  475 ../data/Settling - Telepathics/blocks/ExpertPlus.pkl
#  476 ../data/Settling - Telepathics/blocks/Expert.pkl
#  477 ../data/Settling - Telepathics/blocks/Hard.pkl
#  478 ../data/Eon Break/blocks/ExpertPlus.pkl
#  479 ../data/Eon Break/blocks/Expert.pkl
#  480 ../data/Eon Break/blocks/Hard.pkl
#  481 ../data/Hometown/blocks/ExpertPlus.pkl
#  482 ../data/Seductive Robot - Waveshaper/blocks/Expert.pkl
#  483 ../data/Crypteque/blocks/Expert.pkl
#  484 ../data/Ber Zer Ker/blocks/Expert.pkl
#  485 ../data/Final Song/blocks/Expert.pkl
#  486 ../data/Final Song/blocks/Hard.pkl
#  487 ../data/Professional Griefers/blocks/Expert.pkl
#  488 ../data/Carol of the Bells - Kim Walker-Smith/blocks/Expert.pkl
#  489 ../data/Carol of the Bells - Kim Walker-Smith/blocks/Hard.pkl
#  490 ../data/Ghostbusters/blocks/Expert.pkl
#  491 ../data/Ghostbusters/blocks/Hard.pkl
#  492 ../data/Kaizo Trap/blocks/ExpertPlus.pkl
#  493 ../data/Kaizo Trap/blocks/Expert.pkl
#  494 ../data/Kaizo Trap/blocks/Hard.pk

#  637 ../data/EC1-Haunted/blocks/Expert.pkl
#  638 ../data/EC1-Haunted/blocks/Hard.pkl
#  639 ../data/Dramophone/blocks/Hard.pkl
#  640 ../data/Last Dance (Custic Remix)/blocks/ExpertPlus.pkl
#  641 ../data/Last Dance (Custic Remix)/blocks/Expert.pkl
#  642 ../data/Last Dance (Custic Remix)/blocks/Hard.pkl
#  643 ../data/Permanent/blocks/ExpertPlus.pkl
#  644 ../data/Permanent/blocks/Expert.pkl
#  645 ../data/Gillette/blocks/Expert.pkl
#  646 ../data/Kyle's Mom's a Bitch/blocks/ExpertPlus.pkl
#  647 ../data/Protectors of the Earth/blocks/Expert.pkl
#  648 ../data/Protectors of the Earth/blocks/Hard.pkl
#  649 ../data/Twisted Transistor/blocks/Expert.pkl
#  650 ../data/You're Welcome/blocks/Hard.pkl
#  651 ../data/Chocolate Planet/blocks/ExpertPlus.pkl
#  652 ../data/Chocolate Planet/blocks/Expert.pkl
#  653 ../data/Chocolate Planet/blocks/Hard.pkl
#  654 ../data/DDH/blocks/Expert.pkl
#  655 ../data/DDH/blocks/Hard.pkl
#  656 ../data/Just Be Friends/blocks/Expert.pkl
#  657 ../data/Jus

#  796 ../data/Shut Up and Dance/blocks/ExpertPlus.pkl
#  797 ../data/Shut Up and Dance/blocks/Expert.pkl
#  798 ../data/Shut Up and Dance/blocks/Hard.pkl
#  799 ../data/Burn/blocks/Expert.pkl
#  800 ../data/Burn/blocks/Hard.pkl
#  801 ../data/The Island Part 2 Dusk/blocks/Expert.pkl
#  802 ../data/Halloweekend/blocks/Expert.pkl
#  803 ../data/Halloweekend/blocks/Hard.pkl
#  804 ../data/Hyadain no Joujou Yuujou/blocks/ExpertPlus.pkl
#  805 ../data/Hyadain no Joujou Yuujou/blocks/Expert.pkl
#  806 ../data/Hyadain no Joujou Yuujou/blocks/Hard.pkl
#  807 ../data/High Hopes/blocks/Expert.pkl
#  808 ../data/Il faut tenter de vivre/blocks/ExpertPlus.pkl
#  809 ../data/Another Heaven/blocks/Expert.pkl
#  810 ../data/Another Heaven/blocks/Hard.pkl
#  811 ../data/Komiya Mao - (can you) understand me/blocks/Expert.pkl
#  812 ../data/1My Truth/blocks/Hard.pkl
#  813 ../data/Something Strange - Vicetone/blocks/Expert.pkl
#  814 ../data/Something Strange - Vicetone/blocks/Hard.pkl
#  815 ../data/Fi

#  963 ../data/My Soul,Your Beats/blocks/Hard.pkl
#  964 ../data/BTH 4 - So Fabulous!!/blocks/ExpertPlus.pkl
#  965 ../data/Bob Ross - Happy Little Clouds/blocks/Hard.pkl
#  966 ../data/Phil Collins - In the Air Tonight/blocks/Expert.pkl
#  967 ../data/Phil Collins - In the Air Tonight/blocks/Hard.pkl
#  968 ../data/BUBBLE TEA - dark cat/blocks/Expert.pkl
#  969 ../data/BUBBLE TEA - dark cat/blocks/Hard.pkl
#  970 ../data/Midnight City/blocks/ExpertPlus.pkl
#  971 ../data/Midnight City/blocks/Expert.pkl
#  972 ../data/Midnight City/blocks/Hard.pkl
#  973 ../data/Fall Out Boy - My song know what you did in the dark/blocks/Expert.pkl
#  974 ../data/Fall Out Boy - My song know what you did in the dark/blocks/Hard.pkl
#  975 ../data/EGOIST - RELOADED by Saut/blocks/ExpertPlus.pkl
#  976 ../data/EGOIST - RELOADED by Saut/blocks/Expert.pkl
#  977 ../data/EGOIST - RELOADED by Saut/blocks/Hard.pkl
#  978 ../data/Professor Pumplestickle/blocks/ExpertPlus.pkl
#  979 ../data/So What/blocks/Hard.p

# 1118 ../data/Circus/blocks/ExpertPlus.pkl
# 1119 ../data/Circus/blocks/Expert.pkl
# 1120 ../data/Circus/blocks/Hard.pkl
# 1121 ../data/Kamikaze/blocks/Expert.pkl
# 1122 ../data/Kamikaze/blocks/Hard.pkl
# 1123 ../data/BRAVE JEWEL (TV size)/blocks/ExpertPlus.pkl
# 1124 ../data/BRAVE JEWEL (TV size)/blocks/Expert.pkl
# 1125 ../data/BRAVE JEWEL (TV size)/blocks/Hard.pkl
# 1126 ../data/Responsibility Response/blocks/Expert.pkl
# 1127 ../data/Rammstein Du Hast/blocks/Expert.pkl
# 1128 ../data/Rammstein Du Hast/blocks/Hard.pkl
# 1129 ../data/Fallen Kingdom/blocks/Hard.pkl
# 1130 ../data/Peanut Butter Jelly - Galantis/blocks/Expert.pkl
# 1131 ../data/Naru Hakobe/blocks/Expert.pkl
# 1132 ../data/Daft Mouth/blocks/Hard.pkl
# 1133 ../data/Stay The Night/blocks/Expert.pkl
# 1134 ../data/Viva La Vida/blocks/Hard.pkl
# 1135 ../data/Hated By Life/blocks/ExpertPlus.pkl
# 1136 ../data/Rum N' Bass/blocks/ExpertPlus.pkl
# 1137 ../data/Rum N' Bass/blocks/Expert.pkl
# 1138 ../data/Rum N' Bass/blocks/Hard

# 1276 ../data/Crack Traxxxx/blocks/Hard.pkl
# 1277 ../data/Lindsey Stirling - The Arena/blocks/Expert.pkl
# 1278 ../data/Illenium - Fortress/blocks/Hard.pkl
# 1279 ../data/Joyful PrettyCure Christmas/blocks/Expert.pkl
# 1280 ../data/Duality/blocks/ExpertPlus.pkl
# 1281 ../data/Reality Check Through The Skull/blocks/ExpertPlus.pkl
# 1282 ../data/Swamp Thing/blocks/ExpertPlus.pkl
# 1283 ../data/Swamp Thing/blocks/Expert.pkl
# 1284 ../data/Swamp Thing/blocks/Hard.pkl
# 1285 ../data/Tame Impala - Let It Happen/blocks/Expert.pkl
# 1286 ../data/Tame Impala - Let It Happen/blocks/Hard.pkl
# 1287 ../data/Life Will Change poppin party cover/blocks/ExpertPlus.pkl
# 1288 ../data/Life Will Change poppin party cover/blocks/Expert.pkl
# 1289 ../data/Life Will Change poppin party cover/blocks/Hard.pkl
# 1290 ../data/NSP - First Date/blocks/Expert.pkl
# 1291 ../data/Ocean/blocks/Expert.pkl
# 1292 ../data/Ocean/blocks/Hard.pkl
# 1293 ../data/Cut Your Teeth/blocks/Expert.pkl
# 1294 ../data/Cut Your Tee

# 1435 ../data/Bingle Bangle - AOA/blocks/Expert.pkl
# 1436 ../data/Mortal Kombat (Original Theme)/blocks/Expert.pkl
# 1437 ../data/Mortal Kombat (Original Theme)/blocks/Hard.pkl
# 1438 ../data/Back in Black/blocks/Expert.pkl
# 1439 ../data/Back in Black/blocks/Hard.pkl
# 1440 ../data/The Resistance/blocks/Expert.pkl
# 1441 ../data/The Resistance/blocks/Hard.pkl
# 1442 ../data/UTDummy/blocks/Expert.pkl
# 1443 ../data/UTDummy/blocks/Hard.pkl
# 1444 ../data/ClariS - SHIORI -TV size mix-/blocks/ExpertPlus.pkl
# 1445 ../data/Levels - Avicii/blocks/Expert.pkl
# 1446 ../data/Dangerous Night/blocks/Expert.pkl
# 1447 ../data/Dangerous Night/blocks/Hard.pkl
# 1448 ../data/NSP - Cool Patrol/blocks/Expert.pkl
# 1449 ../data/NSP - Cool Patrol/blocks/Hard.pkl
# 1450 ../data/Red Like Roses fix/blocks/Expert.pkl
# 1451 ../data/High on Life (feat. Bonn)/blocks/Expert.pkl
# 1452 ../data/High on Life (feat. Bonn)/blocks/Hard.pkl
# 1453 ../data/Mans Not Hot/blocks/Expert.pkl
# 1454 ../data/Uso no Hibana/

In [30]:
X

shifted  \
name                                          time                                                            
../data/Shake That - Eminem/blocks/Expert.pkl 6.970093    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
                                              7.110280    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
                                              7.250467    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
                                              7.390654    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
                                              7.530841    [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
                                              7.671028    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
                                              7.811215    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
                                              7.951402    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
                                              8.091589    [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
                                              8.231776    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
                                              8.371963    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
                                              8.512150    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
                                              8.652336    [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
                                              9.213084    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
                                              9.493458    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
                                              9.773832    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
                                              10.054206   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
                                              10.614953   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
                                              11.175701   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
                                              11.736449   [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
                                              13.138318   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
                                              13.418692   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
                                              13.558879   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
                                              17.343925   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
                                              17.624299   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...   
                                              17.904673   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
                                              18.465421   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
                                              18.745794   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
                                              19.026168   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
                                              19.306542   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
...                                                                                                     ...   
../data/Calling/blocks/Hard.pkl               107.783226  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
                                              108.170323  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
                                              108.363871  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
                                              108.557419  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

In [36]:
def pp(row):
    print('*' * 69)
    print(row)
    
def round_up(num:float, prec: int) -> int:
    return int(math.ceil((10 ** prec) * num) / (10 ** prec))
    
def get_len_category(song):
    return round_up(len(song), -2)

def get_mask(X):
    mask = X.groupby('name').apply(get_len_category)
    return mask.to_dict()

In [37]:
grouped = X.groupby(get_mask(X), level=0)

for name, group in grouped:
    print(f'{name}: {len(group.groupby("name").groups)}')

100: 5
200: 25
300: 105
400: 244
500: 312
600: 264
700: 184
800: 131
900: 74
1000: 49
1100: 31
1200: 16
1300: 4
1400: 6
1500: 5
1600: 3
1700: 1
1800: 1
2300: 1
2600: 1


In [39]:
def create_batch(group, ceil_len):
    print(f'Creating batch of ceil_len {ceil_len:6} with {len(group)} rows')
    
    batch = []
    for name, song in group.groupby('name'):
#         print(f'{l} {name}')
        empty_row = song.tail(1).squeeze().apply(np.zeros_like)
        df_to_add = pd.DataFrame([empty_row] * (ceil_len - len(song)))
        batch.append(pd.concat([song, df_to_add]))

    return pd.concat(batch)

In [40]:
def list2numpy(batch, col_name):
    return np.array(batch.groupby('name')[col_name].apply(list).to_list())
#     return np.array(batch[col_name].to_list())

def train_generator(df):
    grouped = X.groupby(get_mask(df), level=0)
    batches = [create_batch(group, ceil_len) for ceil_len, group in grouped]
    batches = [batch for batch in batches if len(batch.groupby('name').groups) > 8]
    
    while True:
        for batch in batches:
            yield [list2numpy(batch, 'shifted'), list2numpy(batch, 'times')], list2numpy(batch, 'output')

In [41]:
list2numpy(X, 'shifted')

### Concatering model

In [ ]:
from keras.layers import Dense, LSTM, Flatten, Input, Activation, TimeDistributed, concatenate
from keras.models import Sequential
from keras.models import Model

In [ ]:
def get_model():
    in1 = Input(shape=(None, 216)) # last blocks
    in2 = Input(shape=(None, 2))   # times
    
    x1 = TimeDistributed(Dense(50, activation='elu'))(in1)
    x2 = TimeDistributed(Dense(3, activation='elu'))(in2)
    
    out = concatenate([x1, x2], axis=-1)
    out = LSTM(50, return_sequences=True)(out)
    out = TimeDistributed(Dense(216, activation='sigmoid'))(out)

    model = Model(inputs=[in1, in2], outputs=out)
    
    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy',
                  metrics=['accuracy', 'binary_crossentropy'])
    
    return model

In [ ]:
model = get_model()
model.summary()

In [26]:
model = get_model()

In [34]:
for x, y in train_generator(X):
    model.fit(x, y, batch_size=32, validation_split=0.1)

# model.predict()

NameError: name 'get_mask' is not defined

In [28]:
# def train_generator():
#     while True:
#         for x1, x2, y in zip(X['blocks'][:300], X['times'][:300], Y[:300]):
#             yield [np.array([x1, ]), np.array([x2, ])], np.array([y, ])
            
def eval_generator():
    while True:
        for x1, x2, y in zip(X['blocks'][300:], X['times'][300:], Y[300:]):
            yield [np.array([x1, ]), np.array([x2, ])], np.array([y, ])
            
model = get_model()
model.batch_size = 8            

model.fit_generator(train_generator(), steps_per_epoch=300, epochs=1, verbose=1, 
                    use_multiprocessing=False)
model.evaluate_generator(eval_generator(), steps=19, )

Epoch 1/1
Creating batch of ceil_len    100 with 98 rows
Creating batch of ceil_len    200 with 2003 rows
Creating batch of ceil_len    300 with 13946 rows
Creating batch of ceil_len    400 with 45199 rows
Creating batch of ceil_len    500 with 68740 rows
Creating batch of ceil_len    600 with 68669 rows
Creating batch of ceil_len    700 with 54062 rows
Creating batch of ceil_len    800 with 54191 rows
Creating batch of ceil_len    900 with 30431 rows
Creating batch of ceil_len   1000 with 22357 rows
Creating batch of ceil_len   1100 with 17478 rows
Creating batch of ceil_len   1200 with 9058 rows
Creating batch of ceil_len   1300 with 2487 rows
Creating batch of ceil_len   1400 with 2712 rows
Creating batch of ceil_len   1600 with 3031 rows
Creating batch of ceil_len   1800 with 1729 rows
Creating batch of ceil_len   2300 with 2219 rows
Creating batch of ceil_len   2600 with 2592 rows
  6/300 [..............................] - ETA: 10:55 - loss: 0.6652 - acc: 0.7613 - binary_crossentr

KeyboardInterrupt: 

# Multi input test

In [ ]:
from keras.layers import Dense ,LSTM,concatenate,Input,Flatten
import tensorflow as tf
import matplotlib
import numpy as np
import pandas as pd
from keras.models import Model
from keras.callbacks import EarlyStopping
#
def model(i,p,data_a,data_b,labels):
#     #x=tf.placeholder(tf.float32,shape=(i,1,None))
#     x=Input(shape=(None, 2))
#     #x = Flatten()(x)
#     y=Input(shape=(None, 4))
#     #y = Flatten()(y)
#     #y=tf.placeholder(tf.float32,shape=(i,1,None))
#     #LSTM layers
#     admi=LSTM(40,return_sequences=True)(x)
#     pla=LSTM(40,return_sequences=True)(y)
#     out=concatenate([admi,pla],axis=-1)
#     print(pla)
#     #out=Flatten(input_shape=(None,1))(out)
#     print(out)
#     #out=np.reshape(out,(1,9))
#     output=TimeDistributed(Dense(1, activation='sigmoid'))(out)
#     model = Model(inputs=[x, y], outputs=output)
#     model.compile(optimizer='rmsprop',
#               loss='binary_crossentropy',
#               metrics=['accuracy'])
    n=model.fit([data_a, data_b],labels,batch_size=1, epochs=10)
    return n

# data_a=np.array([300,455,350,560,700,800,200,250,300])
# labes=np.array([455,350,560,700,800,200,250,300,350])
# data_b=np.array([200,255,350,470,600,300,344,322,300])
# data_a=np.reshape(data_a,(9,1,1))
# data_b=np.reshape(data_b,(9,1,1))


data_a = np.random.rand(9, 2, 2)
data_b = np.random.rand(9, 2, 4)
labels = np.random.rand(9, 2, 1)

#s=labes.shape()
#labes=np.reshape(labes,(1,9,1))
print(data_a.shape)
print(data_b.shape)
print(labels.shape)

#labes=labes(9,4)
#data_b=tf.placeholder(tf.float32,shape=(6,1))

#labes=tf.placeholder(tf.float32,shape=(6,1))

re = model(1,4,data_a,data_b,labels)
print(re)

In [ ]:
df = pd.DataFrame({'Col1': [10, 20, 15, 30, 45],
...                    'Col2': [13, 23, 18, 33, 48],
...                    'Col3': [17, 27, 22, 37, 52]})
df

In [ ]:
df['Col11'] = pd.Series([np.zeros((206, 1))] * 5)
df['Col11'] = df['Col11'].shift(-1, fill_value=[np.zeros((206, 1))])
df